In [1]:
import pandas as pd
import gdown
import ast
import json
import numpy as np
import requests
from datetime import datetime
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from collections import Counter
!pip install pymorphy2
import pymorphy2
from scipy import stats

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 8.2 MB 11.4 MB/s 


In [196]:
# Загружаем таблицу с гугл-диска
url = '1OxQupfqEt36O2GwnP7gL-2Kq9pdR5Eii'
output = "vacancies.xlsx"
gdown.download('https://drive.google.com/uc?export=download&id=' + url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1OxQupfqEt36O2GwnP7gL-2Kq9pdR5Eii
To: /content/vacancies.xlsx
173MB [00:01, 114MB/s]


'vacancies.xlsx'

In [197]:
df = pd.read_excel("vacancies.xlsx")

In [198]:
# Переводим текст в словари
df['area'] = df['area'].apply(lambda x: ast.literal_eval(x))
df['salary'] = df['salary'].apply(lambda x: ast.literal_eval(x) if not pd.isnull(x) else None)

In [199]:
# Разбиваем словари по столбцам
df['area_id'] = df['area'].apply(lambda x: x['id'])
df['area_name'] = df['area'].apply(lambda x: x['name'])
df['salary_from'] = df['salary'].apply(lambda x: x['from'] if not pd.isnull(x) else None)
df['salary_to'] = df['salary'].apply(lambda x: x['to'] if not pd.isnull(x) else None)
df['salary_currency'] = df['salary'].apply(lambda x: x['currency'] if not pd.isnull(x) else None)
df['salary_gross'] = df['salary'].apply(lambda x: x['gross'] if not pd.isnull(x) else None)

In [6]:
# Получаем все города и их parent_id
req = requests.get('https://api.hh.ru/areas')  # Посылаем запрос к API
data = req.content.decode()  # Декодируем его ответ, чтобы Кириллица отображалась корректно
req.close()
jsonObj = json.loads(data)
cities, countries_name, areas_name = [], [], []
for i in range(len(jsonObj)):
    for area in jsonObj[i]['areas']:
      if area['areas'] == []:
        cities.append(area)
        countries_name.append(jsonObj[i]['name'])
        areas_name.append(area['name'])
      else:
        for city in area['areas']:
          cities.append(city)
          countries_name.append(jsonObj[i]['name'])
          areas_name.append(area['name'])
df_cities = pd.DataFrame(cities)
df_cities['country_name'] = countries_name
df_cities['areas_name'] = areas_name

In [7]:
# Переводим текст в формат даты
df['published_at'] = df['published_at'].apply(lambda x: datetime.strptime(x,  "%Y-%m-%dT%H:%M:%S%z"))

In [8]:
# Переименовываем столбец
df_cities['area_id'] = df_cities['id']
df_cities = df_cities.drop('id', 1)

In [206]:
# Присваиваем каждой строке имя региона и страны
df['areas_name'] = df['area_id'].map(df_cities.set_index('area_id')['areas_name'])
df['country_name'] = df['area_id'].map(df_cities.set_index('area_id')['country_name'])

In [207]:
# Считаем среднюю з/п
df['salary_avg'] = (df['salary_from'] + df['salary_to'])/2

In [209]:
# Для тех вакансий, у которых з/п указана после вычета налогов делаем ее до вычетов (для России и з/п в рублях)
df['salary_avg'][(df['salary_gross'] == False) & (df['country_name'] == 'Россия') & (df['salary_currency'] == 'RUR')] = df['salary_avg']/0.87

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [178]:
# Подгружаем данные с росстата по з/п
df_gks = pd.read_excel('salary_gks.xlsx')

In [210]:
# Считаем средние з/п по всем регионам
x = pd.DataFrame(df[df['country_name'] == 'Россия']['salary_avg'].groupby(df['areas_name']).mean())
x['areas_name'] = x.index

In [211]:
x['salary_gks'] = x['areas_name'].map(df_gks.set_index('areas_name')['salary_avg'])

In [212]:
# Считаем, насколько посчитанная средняя зарплата отличается от данных с росстата за июнь в рублях
(abs(x['salary_gks'] - x['salary_avg'])).mean()

11431.85659290796

In [213]:
# Корреляция
x['salary_gks'].corr(x['salary_avg'])

0.8216230699845609

In [214]:
x = x.dropna()
stats.ttest_ind(x['salary_avg'],x['salary_gks']) # p-value > 0.05, значит принимаем нулевую гипотезу, значит две выборки равны

Ttest_indResult(statistic=1.0959999552166269, pvalue=0.27468562640111677)

In [219]:
# Описательная статистика для з/п
pd.set_option('float_format', '{:.0f}'.format)
df[(df['salary_currency'] == 'RUR') & (df['country_name'] == 'Россия')].describe()[['salary_from', 'salary_to', 'salary_avg']]

,salary_from,salary_to,salary_avg
count,100350,56764,50306
mean,43864,58758,55334
std,30940,44957,38113
min,1,1,1
25%,25200,32000,31609
50%,35000,45000,44868
75%,50000,70000,66092
max,1000000,1700000,1100000


In [11]:
df['published_at'].describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


count                        164916
unique                       133258
top       2021-09-11 15:36:23+03:00
freq                             50
first     2021-08-13 09:16:35+03:00
last      2021-09-13 19:11:47+03:00
Name: published_at, dtype: object

In [220]:
df.count()

id                              164916
premium                         164916
billing_type                    164916
relations                       164916
name                            164916
insider_interview                  708
response_letter_required        164916
area                            164916
salary                          130625
type                            164916
address                          81832
allow_messages                  164916
site                            164916
experience                      164916
schedule                        164916
employment                      164916
department                        9261
contacts                             0
description                     164916
branded_description              24645
vacancy_constructor_template       499
key_skills                      164916
accept_handicapped              164916
accept_kids                     164916
archived                        164916
response_url             

In [13]:
table = df.groupby(['country_name']).size().reset_index(name='count')
table
# table.to_excel('table.xlsx', index = False)

,country_name,count
0,Азербайджан,355
1,Беларусь,12560
2,Грузия,72
3,Другие регионы,976
4,Казахстан,13069
5,Кыргызстан,688
6,Россия,127710
7,Узбекистан,2589
8,Украина,6897


In [14]:
df.groupby(['areas_name']).size().reset_index(name='count')

,areas_name,count
0,Абай,22
1,Абхазия,3
2,Австралия,2
3,Австрия,6
4,Акташ (Узбекистан),2
...,...,...
360,Ямало-Ненецкий АО,2000
361,Янгиер,1
362,Янгиюль,3
363,Япония,7


In [15]:
# Избавляемся от html-кода в столбце 'description'
code = ['li', 'strong', 'p', 'ul', 'quot', 'br', 'em']
for word in code:
  df['description'] = df['description'].str.replace(word, '')

In [16]:
# Считаем наиболее часто встречающиеся слова в описании вакансий, за исключением stopwords и небуквенных символов
s = df['description'].to_string()


In [17]:
allWords = nltk.tokenize.word_tokenize(s)
stopwords = nltk.corpus.stopwords.words('russian')

In [18]:
symbols = ['<', '>', '...', ',', '-', '.', ';', ':', '«', '»', '!', '(', ')', '/', '–', '—', '&']

In [19]:
morph = pymorphy2.MorphAnalyzer()

In [20]:
allWordExceptStopDist = nltk.FreqDist(morph.parse(w.lower())[0].normal_form for w in allWords if w.lower() not in symbols and w.lower() not in stopwords)   

In [21]:
mostCommon = Counter(allWordExceptStopDist).most_common(15)

In [22]:
mostCommon

[('обязанность', 72920),
 ('работа', 17233),
 ('компания', 15665),
 ('требоваться', 6936),
 ('крупный', 6574),
 ('сеть', 6389),
 ('приглашать', 5114),
 ('магазин', 3981),
 ('работать', 3928),
 ('хотеть', 3834),
 ('ооо', 3615),
 ('команда', 3533),
 ('каждый', 3450),
 ('связь', 3432),
 ('приём', 3332)]